<a href="https://colab.research.google.com/github/DiegoAnas/SNN-NLP/blob/master/Sentiment%20LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [2]:
import tensorflow
print(tensorflow.__version__)
import tensorflow.keras as keras

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, \
  concatenate, Dense, Activation, LSTM, Dropout, Bidirectional, Flatten

import numpy as np
from sklearn.model_selection import StratifiedKFold

2.1.0-rc1


In [0]:
# Parameters:
# Embedding
max_features = 20000
maxlen = 400
embedding_size = 128

# LSTM
lstm_output_size = 100

# Training
batch_size = 30
epochs = 3

In [4]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
x_data = np.concatenate((x_train, x_test))
y_data = np.concatenate((y_train, y_test))
skf = StratifiedKFold(n_splits=5, random_state=42)

Loading data...
17465344/17464789 [==============================] - 0s 0us/step
25000 train sequences
25000 test sequences
Pad sequences (samples x time)


In [0]:
# Network Architecture
def create_LSTM_model(lstm_cells:[int], activation:str='tanh', recurrent_activation='sigmoid',
                      dropout:float=0.0, recurrent_dropout:float=0.0, embedding_dropout:bool=False, 
                      embedding_dropout_rate:float=0.0):
  # LSTM 
  var_input = Input(shape=(400,))
  emb_layer = Embedding(max_features,
                      embedding_size,
                      input_length=maxlen)(var_input)
  if embedding_dropout:
    emb_layer = SpatialDropout1D(rate=embedding_dropout_rate)(emb_layer)
  previous_layer = emb_layer
  for idx, lstm_units in enumerate(lstm_cells): 
    lstm_layer = LSTM(units=lstm_units, activation=activation, recurrent_activation=recurrent_activation, 
                    dropout=dropout, recurrent_dropout=recurrent_dropout, 
                    return_sequences= (idx!=len(lstm_cells)-1))(previous_layer)
    previous_layer = lstm_layer
  dense_layer = Dense(1, activation='sigmoid')(previous_layer)
  model = Model(inputs=var_input, outputs=dense_layer)
  model.compile(loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])
  return model

In [7]:
# Test 1-layer LSTM RNN
for train, test in skf.split(x_data, y_data):
  target = create_LSTM_model(activation='tanh', lstm_cells=[lstm_output_size])
  target.fit(x_data[train], y_data[train],
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_data[test], y_data[test]))

Train on 40000 samples, validate on 10000 samples
Epoch 1/3
40000/40000 [==============================] - 86s 2ms/sample - loss: 0.3966 - accuracy: 0.8237 - val_loss: 0.3340 - val_accuracy: 0.8623
Epoch 2/3
40000/40000 [==============================] - 77s 2ms/sample - loss: 0.2275 - accuracy: 0.9132 - val_loss: 0.2976 - val_accuracy: 0.8805
Epoch 3/3
40000/40000 [==============================] - 77s 2ms/sample - loss: 0.1787 - accuracy: 0.9342 - val_loss: 0.3035 - val_accuracy: 0.8829
Train on 40000 samples, validate on 10000 samples
Epoch 1/3
40000/40000 [==============================] - 77s 2ms/sample - loss: 0.4039 - accuracy: 0.8184 - val_loss: 0.3663 - val_accuracy: 0.8417
Epoch 2/3
40000/40000 [==============================] - 76s 2ms/sample - loss: 0.2540 - accuracy: 0.9009 - val_loss: 0.2955 - val_accuracy: 0.8754
Epoch 3/3
40000/40000 [==============================] - 77s 2ms/sample - loss: 0.1728 - accuracy: 0.9359 - val_loss: 0.3194 - val_accuracy: 0.8825
Train on 400

In [8]:
epochs = 2
# Test 1-layer LSTM RNN w SELU
for train, test in skf.split(x_data, y_data):
  target = create_LSTM_model(activation='selu', lstm_cells=[lstm_output_size])
  target.fit(x_data[train], y_data[train],
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_data[test], y_data[test]))

Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 646s 16ms/sample - loss: 0.5212 - accuracy: 0.7501 - val_loss: 0.4467 - val_accuracy: 0.7864
Epoch 2/2
40000/40000 [==============================] - 646s 16ms/sample - loss: 0.3522 - accuracy: 0.8461 - val_loss: 0.4295 - val_accuracy: 0.8043
Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 635s 16ms/sample - loss: nan - accuracy: 0.5266 - val_loss: nan - val_accuracy: 0.5000
Epoch 2/2
40000/40000 [==============================] - 631s 16ms/sample - loss: nan - accuracy: 0.5000 - val_loss: nan - val_accuracy: 0.5000
Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 652s 16ms/sample - loss: nan - accuracy: 0.5579 - val_loss: nan - val_accuracy: 0.5000
Epoch 2/2
 1470/40000 [>.............................] - ETA: 10:01 - loss: nan - accuracy: 0.5049

KeyboardInterrupt: ignored

Changing activation function to SELU drastically increases training time. Accuracy is poor and training function is unable to calculate loss value. We suspect SeLU activation gets unstably too large.

In [17]:
create_LSTM_model(activation='tanh', lstm_cells=[100,100]).summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 400)]             0         
_________________________________________________________________
embedding_13 (Embedding)     (None, 400, 128)          2560000   
_________________________________________________________________
lstm_18 (LSTM)               (None, 400, 100)          91600     
_________________________________________________________________
lstm_19 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 101       
Total params: 2,732,101
Trainable params: 2,732,101
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Test 2-layer LSTM RNN
for train, test in skf.split(x_data, y_data):
  target = create_LSTM_model(activation='tanh', lstm_cells=[100,100])
  target.fit(x_data[train], y_data[train],
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_data[test], y_data[test]))

Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 107s 3ms/sample - loss: 0.4042 - accuracy: 0.8172 - val_loss: 0.4147 - val_accuracy: 0.8074
Epoch 2/2
40000/40000 [==============================] - 106s 3ms/sample - loss: 0.2845 - accuracy: 0.8823 - val_loss: 0.2620 - val_accuracy: 0.8933
Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 107s 3ms/sample - loss: 0.3719 - accuracy: 0.8353 - val_loss: 0.4808 - val_accuracy: 0.7751
Epoch 2/2
40000/40000 [==============================] - 106s 3ms/sample - loss: 0.2778 - accuracy: 0.8885 - val_loss: 0.2704 - val_accuracy: 0.8933
Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 110s 3ms/sample - loss: 0.5236 - accuracy: 0.7179 - val_loss: 0.3293 - val_accuracy: 0.8668
Epoch 2/2
40000/40000 [==============================] - 107s 3ms/sample - loss: 0.2731 - accuracy: 0.887

In [0]:
# Test 2-layer LSTM RNN w SELU
for train, test in skf.split(x_data, y_data):
  target = create_LSTM_model(activation='selu', lstm_cells=[100,100])
  target.fit(x_data[train], y_data[train],
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_data[test], y_data[test]))

In [0]:
# Network Architecture
def create_BLSTM_model(lstm_cells:[int], activation:str='tanh', recurrent_activation='sigmoid',
                      dropout:float=0.0, recurrent_dropout:float=0.0, embedding_dropout:bool=False, 
                      embedding_dropout_rate:float=0.0):
  # LSTM 
  var_input = Input(shape=(400,))
  emb_layer = Embedding(max_features,
                      embedding_size,
                      input_length=maxlen)(var_input)
  if embedding_dropout:
    emb_layer = SpatialDropout1D(rate=embedding_dropout_rate)(emb_layer)
  previous_layer = emb_layer
  for idx, lstm_units in enumerate(lstm_cells): 
    lstm_layer = Bidirectional(LSTM(units=lstm_units, activation=activation, recurrent_activation=recurrent_activation, 
                    dropout=dropout, recurrent_dropout=recurrent_dropout, 
                    return_sequences= (idx!=len(lstm_cells)-1)))(previous_layer)
    previous_layer = lstm_layer
  dense_layer = Dense(1, activation='sigmoid')(previous_layer)
  model = Model(inputs=var_input, outputs=dense_layer)
  model.compile(loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])
  return model

In [21]:
# Test 1-layer BILSTM RNN
for train, test in skf.split(x_data, y_data):
  target = create_BLSTM_model(activation='tanh', lstm_cells=[lstm_output_size])
  target.fit(x_data[train], y_data[train],
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_data[test], y_data[test]))

Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 105s 3ms/sample - loss: 0.4113 - accuracy: 0.8168 - val_loss: 0.3386 - val_accuracy: 0.8646
Epoch 2/2
40000/40000 [==============================] - 106s 3ms/sample - loss: 0.3653 - accuracy: 0.8414 - val_loss: 0.2999 - val_accuracy: 0.8784
Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 106s 3ms/sample - loss: 0.3838 - accuracy: 0.8319 - val_loss: 0.3089 - val_accuracy: 0.8754
Epoch 2/2
40000/40000 [==============================] - 105s 3ms/sample - loss: 0.2899 - accuracy: 0.8816 - val_loss: 0.3591 - val_accuracy: 0.8551
Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 106s 3ms/sample - loss: 0.4190 - accuracy: 0.8125 - val_loss: 0.5821 - val_accuracy: 0.6733
Epoch 2/2
40000/40000 [==============================] - 105s 3ms/sample - loss: 0.2561 - accuracy: 0.899

In [22]:
# Test 1-layer BILSTM RNN w SELU
for train, test in skf.split(x_data, y_data):
  target = create_BLSTM_model(activation='selu', lstm_cells=[lstm_output_size])
  target.fit(x_data[train], y_data[train],
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_data[test], y_data[test]))

Train on 40000 samples, validate on 10000 samples
Epoch 1/2
 1200/40000 [..............................] - ETA: 20:46 - loss: 0.6951 - accuracy: 0.5000

KeyboardInterrupt: ignored